бейзлайн
- первая часть (нахождения новостей с упоминанием компаний)
    - использовать только заголовки (датасет https://github.com/yutkin/Lenta.Ru-News-Dataset)
    - ~находить компании в заголовках через заранее заготовленный список компаний с Московской биржи~
    - сначала попробовать только на Яндексе
- вторая часть (проверять, как конкретная новость повлияла на акции компании)
    - смотреть, изменилась ли цена (на 10%, Яндекс из-за закона Горелкина падал на 15%) на фиксированный процент в тот же день (далее можно попробовать 24 рабочих часа после новости)
    - таким образом сделаем датасет для второй задачи (вида "текст новости - компания - таргет (цена понизилась / осталась прежней / повысилась)")
    - обучить на трейне такого датасета модель, которая получает на вход эмбеддинг предложения и пытается предсказать влияние на акции (в какой-то степени это будет sentiment analysis), вначале можно простенькую модель из нескольких слоев с релу, в конце софтмакс на 3 класса



### Сейчас:
- ~найти, откуда взять данные по бирже~ -- yfinance
- ~посмотреть период закона Горелкина (27.07.19) (нужно проверить наличие тех же проблем, что внизу этой ячейки)~ -- вроде по графикам таких проблем нет
- ~сделать функцию, которая к каждой новости покажет изменение цены в процентах~
    - считать изменение к прошлому дню, если вчерашнее или сегодняшнее значение недоступно, то NaN
- ~засунуть эти значения к соответствующим новостям~
- ~посчитать таргеты (изменения вверх/вниз больше х% (1/-1) или 0)~
- добавить других компаний
    - где-то получить лист русских компаний, который есть в yahoo finance
- обучить модель (предобученную на sentiment analysis, добавить пару слоев сверху и выход из 3 логитов, дальше софтмакс и получаем вероятность класса)

### Проверки:
- что случилось после новостей о законе Горелкина

### Проблемы:
- ~Проблемы с данными от pandas_datareader, в примере ниже слишком часто акции торгуются в нуле
```mr = pandas_datareader.data.DataReader('YNDX', 'moex', start='2019-01-01', end='2019-10-31')
mr.plot(subplots=True, figsize=(5, 10))```~ -- больше не использую pandas datareader, использую yfinance
- ~из-за закона Горелкина (27.07.19) акции упали с 2520 до 2441, но без новостей падают еще больше (27.05.19, 2404 -> 2280)~ -- проблема из-за pandas datareader, больше не актуально

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import dateutil
import datetime
import yfinance as yf
import json
import os
import requests

- взять интерфакс, не только российские компании
- посмтреть глазами, насколько разумны -1 и 1 (и из-за этого возможно поменять параметры)
- достать данные за более короткий срок
- обучить классификатор, ненулевых таргетов нужно 1000 (для предобученных можно меньше)
- разница -15 минут и 4 часа, смотреть разницу между ними, если больше 5-10%, то ненулвой таргет (также можно сравнить минимум-максимум за этот промежуток)

In [2]:
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 2 > /home/student/large_files_data/interfax_small.csv
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 1


In [3]:
companies_to_listing_names = {
    'Яндекс': 'YNDX',
#     'Роснефть': 'ojscy',
#     'Лукойл': 'lukoy',
# #     'Сбербанк': 'SBER.ME',
#     'Сбербанк': 'SBRCY',
# #     'Русал': 'RUAL.ME',
# #     'Русал': '0486.HK',
#     'Газпром': 'OGZPY',
# #     'ВТБ': 'VTBR.ME',
# #     'Магнит': 'MGNT.ME',
# #     'Магнит': 'MGNT.IL',
# #     'Транснефть': 'TRNFP.ME',
# #     'X5 Retail Group': 'FIVE.ME',
# #     'Сургутнефтегаз': 'SGGD.VI',
#     'Сургутнефтегаз': 'SGTPY',
    
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Facebook': 'FB',
    'Amazon': 'AMZN',
    'Netflix': 'NFLX',
}

listing_names_to_companies = {value: key for (key, value) in companies_to_listing_names.items()}

In [4]:
interfax_news_table = pd.read_csv(
    '/home/student/large_files_data/interfax_20160101_20191015.csv',
#     sep=',',
    na_values='-',
    escapechar='\\',
    index_col=False,
    usecols=['date', 'title', 'text']
)

In [5]:
interfax_news_table.head()

,date,title,text
0,2019-09-09 14:09:00,Верховный суд в октябре рассмотрит жалобу на п...,Москва. 9 сентября. INTERFAX.RU - Верховный су...
1,2019-09-10 14:12:00,Суд Петербурга отменил выдворение блогера-оппо...,Москва. 10 сентября. INTERFAX.RU - Петербургск...
2,2019-09-11 14:20:00,Евро опустился ниже 72 рублей впервые со 2 авг...,Москва. 11 сентября. INTERFAX.RU - Евро в ходе...
3,2019-09-12 15:30:00,"ЕЦБ снизил ставку по депозитам до минус 0,5%",Москва. 12 сентября. INTERFAX.RU - Европейский...
4,2019-09-13 13:40:00,Москва негативно отнеслась к словам Зеленского...,Москва. 13 сентября. INTERFAX.RU - В Кремле не...


In [16]:
# Extract date
interfax_news_table['date'] = pd.to_datetime(
    interfax_news_table['date'],
    format='%y-%m-%d %H:%M:%S',
    exact=False
)

In [17]:
# Leave only companies-related news
company_news_table_parts = []
for name in companies_to_listing_names.keys():
    company_news_table = interfax_news_table
    company_news_table = company_news_table[company_news_table.title.str.contains(name, case=False)]
    company_news_table['company'] = name
    company_news_table_parts.append(company_news_table)

interfax_dataset = pd.concat(company_news_table_parts).reset_index(drop=True)
# interfax_dataset.dropna(axis=0, subset=['price_change'], inplace=True)

interfax_dataset.head()

# interfax_dataset = interfax_news_table

/home/student/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date,title,text,company
0,2019-09-24 12:31:00,"""Яндекс"" вернулся в список самых быстрорастущи...","Москва. 24 сентября. INTERFAX.RU - ""Яндекс"" вп...",Яндекс
1,2019-09-24 13:55:00,"Техдиректор ""ВКонтакте"" присоединится к фудтех...",Москва. 24 сентября. INTERFAX.RU - Новым техни...,Яндекс
2,2019-09-12 01:51:00,"""Яндекс"" вложит 5 млрд рублей в подготовку IT-...","Москва. 12 сентября. INTERFAX.RU - ""Яндекс"" об...",Яндекс
3,2019-09-11 09:49:00,"Gett пожаловался в ФАС на сделку ""Яндекс.Такси...","Сделка усилит доминирование сервиса ""Яндекса"" ...",Яндекс
4,2016-01-15 14:24:00,"""Яндекс.переводчик"" освоил язык эльфов",Москва. 15 января. INTERFAX.RU - Ко дню рожден...,Яндекс


In [18]:
len(interfax_dataset)

1414

In [15]:
len(interfax_dataset[interfax_dataset['date'] > '2017-05-31'])

910

In [51]:
dir_path = './tiingo_stocks/'
# company_to_json_df = {}

def add_price_change(dataset, companies_to_listing_names):
    
    def get_close_price(df, time, fill_method):
        return df.iloc[df.index.get_loc(time, method=fill_method)]['close']

    def get_price_change(row):
        utc_date = row['date'] - datetime.timedelta(hours=3)  # Get UTC+0
#         print('utc_date', utc_date)
        listing_name = companies_to_listing_names[row['company']]
        df = company_to_json_df[listing_name].set_index('date')
        left_time_bound = utc_date - datetime.timedelta(hours=1)
        right_time_bound = utc_date + datetime.timedelta(hours=2)
#         print('left_time_bound', left_time_bound)
#         print('right_time_bound', right_time_bound)
#         return df.index.get_loc(left_time_bound, method='nearest')
        try:
            new_price = get_close_price(df, right_time_bound, 'ffill')
            old_price = get_close_price(df, left_time_bound, 'bfill')
        except KeyError:
            return np.nan
#             print(row)
#             print('utc_date', utc_date)
#             print('left_time_bound', left_time_bound)
#             print('right_time_bound', right_time_bound)
#             raise
#         print('right_price', right_price)
#         print('left_price', left_price)
        return (new_price - old_price) / old_price * 100
               

    def download_data(company_name):
        url = f'https://api.tiingo.com/iex/{company_name}/prices'
        params = {
            'startDate': '2015-01-01',
            # 'endDate': '2017-12-12',
            'resampleFreq': '60min',
            'token': os.environ.get('TIINGO_TOKEN', None)
        }

        r = requests.get(url=url, params=params)
        r.raise_for_status()
        data = r.json()

        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'w') as f:
            f.write(json.dumps(data))

    company_to_json_df = {}
    for listing_name in companies_to_listing_names.values():
        filename = dir_path + f'{listing_name}.json'
        if not os.path.isfile(filename):
            download_data(listing_name)
        # load json data to df
        company_to_json_df[listing_name] = pd.read_json(filename)#.set_index('date')
#         company_to_json_df[listing_name] = company_to_json_df[listing_name].set_index('date')

#     dataset = dataset.set_index('date')
    return dataset.apply(get_price_change, axis=1)

interfax_dataset_price_change = interfax_dataset
interfax_dataset_price_change['price_change'] = add_price_change(interfax_dataset_price_change, companies_to_listing_names)
interfax_dataset_price_change = interfax_dataset_price_change.dropna()

In [54]:
interfax_dataset_price_change[abs(interfax_dataset_price_change['price_change']) > 5]

,date,title,text,company,price_change
112,2017-07-13 14:48:00,Uber получит 3 из 7 мест в совете директоров с...,Москва. 13 июля. INTERFAX.RU - Представители U...,Яндекс,-13.263525
115,2017-07-13 13:16:00,"""Яндекс.Такси"" присоединило российский бизнес ...","Uber и ""Яндекс"" инвестируют соответственно $22...",Яндекс,-13.263525
116,2017-07-13 13:57:00,"Акции ""Яндекса"" подскочили на 17,5% на новостя...","Москва. 13 июля. INTERFAX.RU - Акции ""Яндекса""...",Яндекс,-13.263525
229,2018-10-31 03:49:00,"""Яндекс"" разработает механизм защиты от поглощ...","Компания учредит специальный фонд, в который в...",Яндекс,-5.521262
242,2018-10-19 14:36:00,"""Яндекс"" подпишет антипиратское соглашение с п...","Стороны договорились о подписании меморандума,...",Яндекс,6.462094
243,2018-10-19 10:03:00,"Капитализация ""Яндекса"" обрушилась на 18% по и...",Москва. 19 октября. INTERFAX.RU - Появление ин...,Яндекс,6.462094
244,2018-10-19 10:30:00,"Акции ""Яндекса"" подешевели на 10% на открытии ...",Снижение капитализации компании спровоцировали...,Яндекс,6.462094
291,2019-05-28 10:30:00,Рестораны Самары оказались самыми дружелюбными...,В них чаще всего сотрудникам сервиса предлагаю...,Яндекс,7.366906
292,2019-05-28 11:43:00,"СП ""Яндекса"" и Сбербанка не ведет переговоры о...",Москва. 28 мая. INTERFAX.RU - Совместное предп...,Яндекс,7.366906
334,2019-10-11 14:41:00,"Акции ""Яндекса"" обновили минимум с конца февра...","Москва. 11 октября. INTERFAX.RU - Акции ""Яндек...",Яндекс,21.189908


In [57]:
def calc_target(dataset, threshold=5.):
    return \
        (dataset['price_change'] > threshold) * 1 + \
        (dataset['price_change'] < -threshold) * -1

interfax_dataset_target = interfax_dataset_price_change
interfax_dataset_target['target'] = calc_target(interfax_dataset_target)
interfax_dataset_target = interfax_dataset_target.drop(['price_change', 'date'], axis=1)

len(interfax_dataset_target[interfax_dataset_target['target'] != 0])

/home/student/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


26

In [58]:
interfax_dataset_target

,title,text,company,target
0,"""Яндекс"" вернулся в список самых быстрорастущи...","Москва. 24 сентября. INTERFAX.RU - ""Яндекс"" вп...",Яндекс,0
1,"Техдиректор ""ВКонтакте"" присоединится к фудтех...",Москва. 24 сентября. INTERFAX.RU - Новым техни...,Яндекс,0
2,"""Яндекс"" вложит 5 млрд рублей в подготовку IT-...","Москва. 12 сентября. INTERFAX.RU - ""Яндекс"" об...",Яндекс,0
3,"Gett пожаловался в ФАС на сделку ""Яндекс.Такси...","Сделка усилит доминирование сервиса ""Яндекса"" ...",Яндекс,0
101,"""Яндекс.Навигатор"" стал показывать освободивши...","Москва. 6 июня. INTERFAX.RU - Программа ""Яндек...",Яндекс,0
102,"Карта ""Яндекс.Транспорта"" начала показывать св...","Москва. 14 июня. INTERFAX.RU - ""Яндекс.Транспо...",Яндекс,0
103,"Минфин РФ сэкономил с помощью Uber и ""Яндекс.Т...",Москва. 13 июня. INTERFAX.RU - Минфин РФ с пом...,Яндекс,0
104,"""Яндекс"" назвал самый популярный по запросам ф...","Москва. 8 июня. INTERFAX.RU - Московский ФК ""С...",Яндекс,0
105,"""Яндекс"" закроет офисы в Киеве и Одессе из-за ...","Ранее власти Украины запретили доступ к ""Яндек...",Яндекс,0
106,РФПИ и CIC проявили интерес к миноритарному па...,Москва. 1 июня. INTERFAX.RU - Российский фонд ...,Яндекс,0


In [59]:
dataset = interfax_dataset_target

In [60]:
# import deeppavlov

# # model = deeppavlov.build_model(deeppavlov.configs., download=True)
# from deeppavlov.models.bert.bert_classifier import BertClassifierModel
# from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor

# BertPreprocessor(
#     vocab_file='',
#     do_lower_case=True,
    
# )
# BertClassifierModel()

In [61]:
# # !mkdir /home/student/large_files_data/tayga_none_fasttextcbow_300_10_2019
parent_dir = '/home/student/large_files_data/'
# embs_path = parent_dir + 'tayga_none_fasttextcbow_300_10_2019.zip'
# !wget http://vectors.nlpl.eu/repository/11/187.zip -O {embs_path}
# !unzip {embs_path}

In [62]:
from gensim.models import KeyedVectors

unzipped_embs_path = parent_dir + 'tayga_none_fasttextcbow_300_10_2019/model.model'
fasttext_model = KeyedVectors.load(unzipped_embs_path)

In [63]:
from razdel import tokenize
import numpy as np

def get_text_embedding(model, phrase):
    embeddings = np.array(
        [model.get_vector(word.text.lower())
         if word.text.lower() in model.vocab
         else np.zeros((model.vector_size,))
         for word in tokenize(phrase)]
    )
    return np.mean(embeddings, axis=0)
    
get_text_embedding(fasttext_model, "В Москве нашли")

array([ 0.00329952,  0.51636818,  0.97818004, -0.11630348, -0.39384873,
       -0.46543727,  0.83378595, -0.14435672, -0.58878418,  0.42940439,
       -0.14867071, -0.65484536,  0.38516421, -0.22604636, -0.93857584,
       -1.19373699,  0.1741313 ,  0.43502013, -0.46075863,  0.01961855,
       -0.37752449, -0.11394024, -0.00377275,  1.04324371, -0.26884733,
       -0.05669444, -0.27271218, -0.45064789,  0.22432417,  0.35779246,
        0.83414837, -0.40738845, -0.06853915,  0.34902072, -0.20284742,
        0.27207336, -0.03016949, -0.26317211, -0.05784377,  0.20304414,
       -0.58317592,  0.45496575, -0.04602315,  1.24028061, -0.0127931 ,
       -0.04771876,  0.78404585,  0.21724252, -0.17761149,  0.40016204,
        0.9719491 ,  0.24062326, -0.71946192, -1.1329535 ,  0.7142989 ,
       -0.35866638, -0.62812591, -0.43037223, -1.07479445,  0.4238079 ,
        0.49084737,  0.75763081,  0.46227547,  1.63199985,  1.44336744,
        0.24074431, -0.70831131,  1.20255784, -0.74403954, -0.10

In [64]:
y = dataset['target'].to_numpy()
X = np.zeros((dataset.shape[0], fasttext_model.vector_size))
for i, title in enumerate(dataset['title']):
    X[i, :] = get_text_embedding(fasttext_model, title)

In [65]:
X.shape

(909, 300)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=0,
    stratify=y  # to keep targets proportion in train and test
)

In [67]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

clf = MLPClassifier()
clf.fit(X_train, y_train)

y_predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.97      1.00      0.99       177
           1       0.00      0.00      0.00         2

    accuracy                           0.97       182
   macro avg       0.32      0.33      0.33       182
weighted avg       0.95      0.97      0.96       182



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
from sklearn.preprocessing import LabelBinarizer

y_train_onehot = LabelBinarizer().fit_transform(y_train)
y_test_onehot = LabelBinarizer().fit_transform(y_test)

In [69]:
import keras
import keras.layers as L

keras_model = keras.Sequential()
keras_model.add(L.Dense(64, activation='relu', input_dim=fasttext_model.vector_size))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(64, activation='relu'))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(3, activation='softmax'))

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
keras_model.compile(loss='categorical_crossentropy',
                    optimizer=sgd,
                    metrics=['accuracy'])

keras_model.fit(X_train,
          y_train_onehot,
          epochs=20,
          batch_size=128,
          class_weight={0: 100, 1: 1, 2: 100})
score = keras_model.evaluate(X_test, y_test_onehot, batch_size=128)

Using TensorFlow backend.


Epoch 1/20
727/727 [==============================] - 0s 380us/step - loss: 6.7444 - acc: 0.1362
Epoch 2/20
727/727 [==============================] - 0s 55us/step - loss: 4.9217 - acc: 0.2393
Epoch 3/20
727/727 [==============================] - 0s 46us/step - loss: 5.0937 - acc: 0.0564
Epoch 4/20
727/727 [==============================] - 0s 43us/step - loss: 7.1816 - acc: 0.1596
Epoch 5/20
727/727 [==============================] - 0s 46us/step - loss: 7.1376 - acc: 0.1348
Epoch 6/20
727/727 [==============================] - 0s 53us/step - loss: 18.9803 - acc: 0.1499
Epoch 7/20
727/727 [==============================] - 0s 56us/step - loss: 15.3064 - acc: 0.1334
Epoch 8/20
727/727 [==============================] - 0s 29us/step - loss: 16.2497 - acc: 0.2105
Epoch 9/20
727/727 [==============================] - 0s 43us/step - loss: 22.1390 - acc: 0.2215
Epoch 10/20
727/727 [==============================] - 0s 53us/step - loss: 25.4460 - acc: 0.2916
Epoch 11/20
727/727 [============

In [70]:
print('Accuracy:', score[1])

Accuracy: 0.021978022018959233
